In [ ]:
#ETAPA 1: Limpieza y preparación de datos

In [1]:
import pandas as pd
import time
covid_febrero = pd.DataFrame()
# --- CONFIGURACIÓN ---
cols_a_cargar = [
    'Province_State', 'Country_Region', 'Last_Update', 'Lat', 'Long_',
    'Confirmed', 'Deaths', 'Recovered', 'Active', 'Combined_Key',
    'Incident_Rate', 'Case_Fatality_Ratio'
]

dtypes_opt = {
    'Confirmed': 'float32', 'Deaths': 'float32', 'Recovered': 'float32',
    'Active': 'float32', 'Incident_Rate': 'float32', 'Case_Fatality_Ratio': 'float32'
}

# --- FUNCIÓN GENERADORA ---
def leer_archivos_por_chunks(mes, dias):
    for i in range(1, dias + 1):
        dia = f"{i:02d}"
        url = f"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{mes}-{dia}-2021.csv"
        try:
            # Leemos el archivo del día
            df_chunk = pd.read_csv(url, usecols=cols_a_cargar, dtype=dtypes_opt)
            # Esto es lo que hace eficiente a chunksize: limpiar trozos pequeños
            df_chunk.columns = df_chunk.columns.str.lower().str.replace(' ', '_')
            # Devolvemos este trocito ya limpio
            yield df_chunk
        except Exception:
            continue
# --- EJECUCIÓN ---
start = time.time()

# Creamos el iterador
iterador_chunks = leer_archivos_por_chunks(mes="02", dias=28)

# concat puede tomar un iterador y lo va consumiendo poco a poco
covid_febrero = pd.concat(iterador_chunks, ignore_index=True)

# --- LIMPIEZA FINAL (Lo que no se pudo hacer por partes) ---
covid_febrero['last_update'] = pd.to_datetime(covid_febrero['last_update'])

# Conversión a categorías
cols_cat = ['country_region', 'province_state']
for col in cols_cat:
    covid_febrero[col] = covid_febrero[col].astype('category')

# Llenar NaNs y convertir a int
cols_num = ['confirmed', 'deaths', 'recovered', 'active']
covid_febrero[cols_num] = covid_febrero[cols_num].fillna(0).astype('int32')

# Calcular columna calculada
covid_febrero['active_cases'] = (
    covid_febrero['confirmed'] - covid_febrero['deaths'] - covid_febrero['recovered']
)

print(f"Tiempo total: {time.time() - start:.2f} seg")
display(covid_febrero.head())

Tiempo total: 9.57 seg


,province_state,country_region,last_update,lat,long_,confirmed,deaths,recovered,active,combined_key,incident_rate,case_fatality_ratio,active_cases
0,NaN,Afghanistan,2021-02-02 05:22:49,33.93911,67.709953,55059,2404,47723,4932,Afghanistan,141.436798,4.366225,4932
1,NaN,Albania,2021-02-02 05:22:49,41.15330,20.168300,78992,1393,47922,29677,Albania,2744.874512,1.763470,29677
2,NaN,Algeria,2021-02-02 05:22:49,28.03390,1.659600,107578,2894,73530,31154,Algeria,245.325974,2.690141,31154
3,NaN,Andorra,2021-02-02 05:22:49,42.50630,1.521800,9972,101,9206,665,Andorra,12906.231445,1.012836,665
4,NaN,Angola,2021-02-02 05:22:49,-11.20270,17.873900,19829,466,18180,1183,Angola,60.332375,2.350093,1183


In [ ]:
#1.1 visualizar los primeros 5 registros del archivo 01-22-2020.csv.
display(covid_febrero.head(5))

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,Afghanistan,2021-02-02 05:22:49,33.939110,67.709953,55059.0,2404.0,47723.0,4932.0,Afghanistan,141.436798,4.366225
1,NaN,Albania,2021-02-02 05:22:49,41.153301,20.168301,78992.0,1393.0,47922.0,29677.0,Albania,2744.874512,1.763470
2,NaN,Algeria,2021-02-02 05:22:49,28.033899,1.659600,107578.0,2894.0,73530.0,31154.0,Algeria,245.325974,2.690141
3,NaN,Andorra,2021-02-02 05:22:49,42.506302,1.521800,9972.0,101.0,9206.0,665.0,Andorra,12906.231445,1.012836
4,NaN,Angola,2021-02-02 05:22:49,-11.202700,17.873899,19829.0,466.0,18180.0,1183.0,Angola,60.332375,2.350093


In [ ]:
#2. Mostrar el número total de filas y columnas del DataFrame.
print(f"Numero total de filas {covid_febrero.shape[0]}\n")
print(f"Numero total de columnas {covid_febrero.shape[1]}\n")

Numero total de filas 112392

Numero total de columnas 12



In [ ]:
#3. Describir los tipos de datos (dtypes) y convertir las columnas necesarias (por ejemplo,fechas).
display(covid_febrero.dtypes)

covid_febrero["Last_Update"] = pd.to_datetime(covid_febrero["Last_Update"])
display(covid_febrero.dtypes)

In [ ]:
#4. Detectar y mostrar valores nulos o faltantes por columna.
print("Cantidad de valores nulos por columna \n")
display(covid_febrero.isnull().sum())

In [ ]:
#5. Eliminar columnas irrelevantes (por ejemplo, códigos FIPS o coordenadas si no se usarán).

#No se eliminan porque ya se hizo al inicio

#covid_febrero = covid_febrero.drop(columns=['FIPS','Admin2'])
#covid_febrero.head()

# parte 5  se hizo al inicio

In [2]:
#6. Estandarizar nombres de columnas (usar formato snake_case).
covid_febrero.columns = covid_febrero.columns.str.lower().str.replace(' ', '_') # Estandarizar los nombres de columnas
display(covid_febrero.columns)
# Como los nombres de países se repiten mucho, 'category' es ideal. #optimizacion
covid_febrero['country_region'] = covid_febrero['country_region'].astype('category')
covid_febrero['province_state'] = covid_febrero['province_state'].astype('category')

Index(['province_state', 'country_region', 'last_update', 'lat', 'long_',
       'confirmed', 'deaths', 'recovered', 'active', 'combined_key',
       'incident_rate', 'case_fatality_ratio', 'active_cases'],
      dtype='object')

In [5]:
#7. Homogeneizar nombres de países (ej. “US” → “United States”).
# Mapeo de Países (Usa .replace con diccionario es eficiente)
country_mapping = {
    "US": "United States", "Korea, North": "North Korea",
    "Korea, South": "South Korea", "Taiwan*": "Taiwan"
}
covid_febrero['country_region'] = covid_febrero['country_region'].replace(country_mapping)

display(covid_febrero['country_region'].unique())


['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', ..., 'Winter Olympics 2022', 'Antarctica', 'North Korea', 'Nauru', 'Tuvalu']
Length: 201
Categories (201, object): ['Afghanistan', 'Albania', 'Algeria', 'Andorra', ..., 'Winter Olympics 2022', 'Yemen', 'Zambia', 'Zimbabwe']

In [6]:
#8. Convertir la columna last_update al formato YYYY-MM-DD.
covid_febrero["last_update"] = pd.to_datetime(covid_febrero["last_update"]).dt.strftime('%Y-%m-%d')
covid_febrero["last_update"] = pd.to_datetime(covid_febrero["last_update"])
display(covid_febrero.head())
covid_febrero.dtypes

,province_state,country_region,last_update,lat,long_,confirmed,deaths,recovered,active,combined_key,incident_rate,case_fatality_ratio,active_cases
0,NaN,Afghanistan,2021-02-02,33.93911,67.709953,55059,2404,47723,4932,Afghanistan,141.436798,4.366225,4932
1,NaN,Albania,2021-02-02,41.15330,20.168300,78992,1393,47922,29677,Albania,2744.874512,1.763470,29677
2,NaN,Algeria,2021-02-02,28.03390,1.659600,107578,2894,73530,31154,Algeria,245.325974,2.690141,31154
3,NaN,Andorra,2021-02-02,42.50630,1.521800,9972,101,9206,665,Andorra,12906.231445,1.012836,665
4,NaN,Angola,2021-02-02,-11.20270,17.873900,19829,466,18180,1183,Angola,60.332375,2.350093,1183


province_state               category
country_region               category
last_update            datetime64[ns]
lat                           float64
long_                         float64
confirmed                       int32
deaths                          int32
recovered                       int32
active                          int32
combined_key                   object
incident_rate                 float32
case_fatality_ratio           float32
active_cases                    int32
dtype: object

In [7]:
#9. Crear una columna active_cases = confirmed - deaths - recovered. #ineficiente
covid_febrero['active_cases'] = covid_febrero['confirmed'] - covid_febrero['deaths'] - covid_febrero['recovered']
display(covid_febrero)

,province_state,country_region,last_update,lat,long_,confirmed,deaths,recovered,active,combined_key,incident_rate,case_fatality_ratio,active_cases
0,NaN,Afghanistan,2021-02-02,33.93911,67.709953,55059,2404,47723,4932,Afghanistan,141.436798,4.366225,4932
1,NaN,Albania,2021-02-02,41.15330,20.168300,78992,1393,47922,29677,Albania,2744.874512,1.763470,29677
2,NaN,Algeria,2021-02-02,28.03390,1.659600,107578,2894,73530,31154,Algeria,245.325974,2.690141,31154
3,NaN,Andorra,2021-02-02,42.50630,1.521800,9972,101,9206,665,Andorra,12906.231445,1.012836,665
4,NaN,Angola,2021-02-02,-11.20270,17.873900,19829,466,18180,1183,Angola,60.332375,2.350093,1183
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112387,Unknown,Ukraine,2021-03-01,NaN,NaN,0,0,0,0,"Unknown, Ukraine",0.000000,0.000000,0
112388,NaN,Nauru,2021-03-01,-0.52280,166.931500,0,0,0,0,Nauru,0.000000,0.000000,0
112389,Niue,New Zealand,2021-03-01,-19.05440,-169.867200,0,0,0,0,"Niue, New Zealand",0.000000,0.000000,0
112390,NaN,Tuvalu,2021-03-01,-7.10950,177.649300,0,0,0,0,Tuvalu,0.000000,0.000000,0


In [8]:
#10. Guardar el DataFrame limpio como covid_clean_enero2020.csv e indicar su tamaño en MB.
import os

covid_febrero.to_csv('covid_clean_febrero_2021.csv', index=False) #para pasar el archivo a csv
covid_febrero = os.path.getsize('covid_clean_febrero_2021.csv') #para obtener el tamaño del archivo.
tamaño_en_mb = covid_febrero / (1024 * 1024) # Convierte el tamaño a MB
print(f"El archivo 'covid clean febrero 2021.csv' ha sido guardado y su tamaño es: {tamaño_en_mb:.2f} MB")

El archivo 'covid clean febrero 2021.csv' ha sido guardado y su tamaño es: 12.86 MB
